# 🧬 Getting Started with EDCA

Welcome! In this tutorial, we will learn how to use EDCA, an AutoML framework that focuses on your data's unique characteristics to build better models.

Instead of just searching for a model, EDCA treats data as a first-class citizen, automatically evolving end-to-end pipelines that clean, reduce, and optimize themselves specifically for your dataset.

## Imports 

In [ ]:
# to import edca from the original src, without installing it via pip
import sys
sys.path.append('../edca')

In [ ]:
import numpy as np
import random
import os
import pandas as pd
import datetime
from edca.evodata import DataCentricAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split # to evaluate the framework
from sklearn import metrics # for calculating metrics

In [ ]:
# EDCA uses randomness, so its it better to fix the seed to ensure always the same results
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

## Load dataset

Start by getting the data to feed EDCA. EDCA receives dataset in the pandas DataFrame format. You can fetch a dataset from OpenML to continue (example 1) or continue with your one dataset (example 2). You need to divide the data into the dataframe (X) and the target series (y).

### Example 1 - fetch a dataset from OpenML

In [ ]:
# example 1 - fetch a classification dataset from OpenML
data_id = 151 # this example used the electricity dataset (https://openml.org/search?type=data&sort=runs&status=active&qualities.NumberOfClasses=gte_2&id=151) 
X, y = fetch_openml(data_id=151, return_X_y=True, as_frame=True)

### Example 2 - your own datasets

Start by telling your dataset's path. Then, load the DataFrame and divided into data and target

In [ ]:
# example 2 - use our own classification datasets
data_path = os.path.join('../data/datasets/Australian.csv')
X = pd.read_csv(data_path) # load dataframe
y = X.pop('class') # divide it into data and target

## Data splitting to later evaluate the EDCA's best solution found

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

## Initialize EDCA

In this example, we will use most EDCA parameters on their default settings. See all the available parameters in `EDCA/edca/README.md`

In [ ]:
# create a folder to store all information regarding the optimization
save_path = '../logs'
os.makedirs(save_path, exist_ok=True)

In [ ]:
# initialize the class
automl = DataCentricAutoML(
    task='classification', # detail the ML task
    seed=SEED, # ensure reproducibility
    metric='f1', # specify the search metric
    time_budget=-1, # specify the time budget in seconds, -1 indicates no time limit, using the iterations as stop criteria
    n_iterations=50, # specify the number of iterations
    log_folder_name=f'{save_path}/experiment-{datetime.datetime.now()}', # specify the log folder to store information
    # search_space_config='classification_models_all.json',
    use_sampling=True, # use sampling to speed up the search
    use_feature_selection=True # use feature selection to speed up the search and improve the model generalization
)

### Optimize the ML pipeline with EDCA

In [ ]:
automl.fit(X_train, y_train)

## Analyze the best solution found

In [ ]:
# the pipeline entire config
automl.pipeline_estimator

In [ ]:
# data processing pipeline
automl.pipeline_estimator.pipeline

In [ ]:
# classification model
automl.pipeline_estimator.model

In [ ]:
# results of the best individual found in the optimization
automl.search_algo.best_fitness_params

In [ ]:
automl.best_individual

### Selected data

Furthermore, we can analyze the selected data in comparison to the original train data. The results show that EDCA substantially reduces the data, which decreases the computational costs associated.

In [ ]:
final_X, final_y = automl.get_final_data()
final_X

EDCA used instance and feature selection, which considerably reduced the final dataset, reducing the computational costs associated.

In [ ]:
print('Original Train dataset:', X_train.shape)
print('EDCA internal train dataset', automl.internal_x_train.shape)
print('EDCA selected dataset:', automl.get_final_data_shape())

## Making predictions

Now, we can use the optimized ML pipeline to make predictions about the saved test data.

In [ ]:
preds = automl.predict(X_test)
preds_proba = automl.predict_proba(X_test)

### Assessing its prediction power

In [ ]:
print(metrics.classification_report(y_test, preds))